Before you turn in the homework, make sure everything runs as expected. To do so, select **Kernel**$\rightarrow$**Restart & Run All** in the toolbar above.  Remember to submit both on **DataHub** and **Gradescope**.

Please fill in your name and include a list of your collaborators below.

In [1]:
NAME = "Dylan Hernandez"
COLLABORATORS = ""

---

# Project 2: NYC Taxi Rides
# Extras

Put all of your extra work in here. Feel free to save figures to use when completing Part 4.

In [2]:
import os
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import zipfile
from sqlalchemy import create_engine
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

sns.set(style="whitegrid", palette="muted")

plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

%matplotlib inline

# Attempt #1

For this model, the features I will use include pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, manhattan, day_of_week, and hour.

In this attempt, I will use the training set from part 1, which is limited to all ride data from the month of January. I will also make sure to handle any outliers in test set by replacing their values with averages (DONE in pt4). 

## Training and Validation

The following code loads the training and validation data from part 1 into a Pandas DataFrame.

In [3]:
# Run this cell to load the data. 
data_file = Path("./", "cleaned_data.hdf")
train_df = pd.read_hdf(data_file, "train")
val_df = pd.read_hdf(data_file, "val")

## Reusable Pipeline for this attempt

In [4]:
# Copied from part 2
def haversine(lat1, lng1, lat2, lng2):
    """
    Compute haversine distance
    """
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    average_earth_radius = 6371
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * average_earth_radius * np.arcsin(np.sqrt(d))
    return h

# Copied from part 2
def manhattan_distance(lat1, lng1, lat2, lng2):
    """
    Compute Manhattan distance
    """
    a = haversine(lat1, lng1, lat1, lng2)
    b = haversine(lat1, lng1, lat2, lng1)
    return a + b

# Copied from part 2
def bearing(lat1, lng1, lat2, lng2):
    """
    Compute the bearing, or angle, from (lat1, lng1) to (lat2, lng2).
    A bearing of 0 refers to a NORTH orientation.
    """
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

# Copied from part 2
def add_time_columns(df):
    """
    Add temporal features to df
    """
    df.is_copy = False # propogate write to original dataframe
    df.loc[:, 'month'] = df['tpep_pickup_datetime'].dt.month
    df.loc[:, 'week_of_year'] = df['tpep_pickup_datetime'].dt.weekofyear
    df.loc[:, 'day_of_month'] = df['tpep_pickup_datetime'].dt.day
    df.loc[:, 'day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
    df.loc[:, 'hour'] = df['tpep_pickup_datetime'].dt.hour
    df.loc[:, 'week_hour'] = df['tpep_pickup_datetime'].dt.weekday * 24 + df['hour']
    return df

# Copied from part 2
def add_distance_columns(df):
    """
    Add distance features to df
    """
    df.is_copy = False # propogate write to original dataframe
    df.loc[:, 'manhattan'] = manhattan_distance(lat1=df['pickup_latitude'],
                                                lng1=df['pickup_longitude'],
                                                lat2=df['dropoff_latitude'],
                                                lng2=df['dropoff_longitude'])

    df.loc[:, 'bearing'] = bearing(lat1=df['pickup_latitude'],
                                   lng1=df['pickup_longitude'],
                                   lat2=df['dropoff_latitude'],
                                   lng2=df['dropoff_longitude'])
    df.loc[:, 'haversine'] = haversine(lat1=df['pickup_latitude'],
                                   lng1=df['pickup_longitude'],
                                   lat2=df['dropoff_latitude'],
                                   lng2=df['dropoff_longitude'])
    return df



def select_columns(data, *columns):
    return data.loc[:, columns]

In [5]:
def process_data_gm2(data, test=False):
    X = (
        data
        
        # Transform data
        .pipe(add_time_columns)
        .pipe(add_distance_columns)
        
        .pipe(select_columns,        
              'pickup_longitude',  
              'pickup_latitude',   
              'dropoff_longitude', 
              'dropoff_latitude',
              'manhattan',
              'day_of_week',
              'hour',
             )
    )
    if test:
        y = None
    else:
        y = data['duration']
        
    return X, y

## Building the Model

In [6]:
# Train
X_train, y_train = process_data_gm2(train_df)
X_val, y_val = process_data_gm2(val_df)
guided_model_2 = lm.LinearRegression(fit_intercept=True)
guided_model_2.fit(X_train, y_train)

# Predict
y_train_pred = guided_model_2.predict(X_train)
y_val_pred = guided_model_2.predict(X_val)

/srv/conda/envs/data100/lib/python3.6/site-packages/pandas/core/generic.py:4388: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/srv/conda/envs/data100/lib/python3.6/site-packages/pandas/core/generic.py:4389: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


In [7]:
#MAE function
def mae(actual, predicted):
    """
    Calculates MAE from actual and predicted values
    Input:
      actual (1D array-like): vector of actual values
      predicted (1D array-like): vector of predicted/fitted values
    Output:
      a float, the MAE
    """
    
    mae = np.mean(np.abs(actual - predicted))
    return mae

In [8]:
assert 200 <= mae(y_val_pred, y_val) <= 300
print("Validation Error: ", mae(y_val_pred, y_val))

Validation Error:  265.554685801


## Reflection
This model can most definitely be improved. Although the validation error for this attempt is in between 200 and 300, my kaggle score 458.44820. Let's try something else...

## Attempt #2

My model in the previous attempt used the a training set that only included data for the month of January. For this attempt, we will use the same features from before,as well as distances, but with data for months Feburary up to May.

Here we will load cleaned training data that includes months up to May.

Let's load all the SQL stuff first

In [9]:
from sqlalchemy import create_engine
from utils import timeit

In [10]:
!ls -lh /srv/db/taxi_2016_student_small.sqlite

-rw-r--r-- 1 root root 2.1G Nov  7 04:43 /srv/db/taxi_2016_student_small.sqlite


In [11]:
DB_URI = "sqlite:////srv/db/taxi_2016_student_small.sqlite"
TABLE_NAME = "taxi"

sql_engine = create_engine(DB_URI)
with timeit():
    print(f"Table {TABLE_NAME} has {sql_engine.execute(f'SELECT COUNT(*) FROM {TABLE_NAME}').first()[0]} rows!")

Table taxi has 15000000 rows!
3.34 s elapsed


Now lets make a cleaned training set that includes months up to May

In [12]:
## Reusing a lot of queries from part 1
q1d_query = f"""
            SELECT *
            FROM {TABLE_NAME}
            WHERE tpep_pickup_datetime
                BETWEEN '2016-02-01' AND '2016-05-01'
                AND record_id % 100 == 0
            ORDER BY tpep_pickup_datetime
            """

In [13]:
q3a_query = f"""
            SELECT *
            FROM ({q1d_query})
            WHERE abs(julianday(tpep_dropoff_datetime) -  julianday(tpep_pickup_datetime)) < 0.5

            """

In [14]:

def bounding_condition(lat_l, lat_u, lon_l, lon_u):
    return f"""
            pickup_longitude <= {lon_u} AND
            pickup_longitude >= {lon_l} AND
            dropoff_longitude <= {lon_u} AND
            dropoff_longitude >= {lon_l} AND
            pickup_latitude <= {lat_u} AND
            pickup_latitude >= {lat_l} AND
            dropoff_latitude <= {lat_u} AND
            dropoff_latitude >= {lat_l} 
            """

q3b_query = f"""
            SELECT *
            FROM ({q3a_query})
            WHERE {bounding_condition(40.63, 40.85, -74.03, -73.75)}

            """
lat_l = 40.63
lat_u = 40.85
lon_l = -74.03
lon_u = -73.75

In [15]:
q3c_query = f"""
            SELECT *
            FROM ({q3b_query})
            WHERE passenger_count > 0

            """
with timeit():
    new_training_set = pd.read_sql_query(q3c_query, sql_engine)
new_training_set['tpep_pickup_datetime'] = pd.to_datetime(new_training_set['tpep_pickup_datetime'])
new_training_set['tpep_dropoff_datetime'] = pd.to_datetime(new_training_set['tpep_dropoff_datetime'])
new_training_set['duration'] = new_training_set["tpep_dropoff_datetime"]-new_training_set["tpep_pickup_datetime"]
new_training_set['duration'] = new_training_set['duration'].dt.total_seconds()
new_training_set = new_training_set[new_training_set['duration'] < 12 * 3600]

3.26 s elapsed


## Training and Validation Split

In [16]:
from sklearn.model_selection import train_test_split
all_m_train_df, all_m_val_df = train_test_split(new_training_set, test_size=0.2, random_state=42)

In [17]:
## Save the data so we can access it in pt4 notebook
Path("data/allmonthdata").mkdir(parents=True, exist_ok=True)
data_file = Path("data/allmonthdata", "allmonth_cleaned_data.hdf") # Path of hdf file
all_m_train_df.to_hdf(data_file, "train2") # Train data of hdf file
all_m_val_df.to_hdf(data_file, "val2") # Val data of hdf file

Now let's build the model!

In [18]:
def remove_manhattan_hav_and_bearing_outliers(df):
    """
    Remove any outliers (3 sds above mean) in manhattan, haversine and bearing column.
    """ 
    manhattan_avg = df.loc[:,'manhattan'].mean()
    manhattan_sd = df.loc[:,'manhattan'].std()
    df['z_scores'] = pd.Series([(i-manhattan_avg)/manhattan_sd for i in df['manhattan']])
    
    haversine_avg = df.loc[:,'haversine'].mean()
    haversine_sd = df.loc[:,'haversine'].std()
    df['z_scores_hav'] = pd.Series([(i-haversine_avg)/haversine_sd for i in df['haversine']])
    
    bearing_avg = df.loc[:,'bearing'].mean()
    bearing_sd = df.loc[:,'bearing'].std()
    df['z_scores_bea'] = pd.Series([(i-bearing_avg)/bearing_sd for i in df['bearing']])
    df = df[(df['z_scores'] > -3) & (df['z_scores'] < 3) & (df['z_scores_hav'] > -3) & (df['z_scores_hav'] < 3) & (df['z_scores_bea'] > -3) & (df['z_scores_bea'] < 3)]
    return df
    

In [19]:
def process_data_gm3(data, test=False):
    if test:
        y = None    
        X = (
            data
        
            # Transform data
            .pipe(add_time_columns)
            .pipe(add_distance_columns)

        
            .pipe(select_columns, 
                  'passenger_count',
                  'pickup_longitude',  
                  'pickup_latitude',   
                  'dropoff_longitude', 
                  'dropoff_latitude',
                  'month',
                  'day_of_week',
                  'hour',
                  'manhattan',
                  'haversine',
                  'bearing'
                 )
        )
    else:
        X = (
            data
        
            # Transform data
            .pipe(add_time_columns)
            .pipe(add_distance_columns)
            .pipe(remove_manhattan_hav_and_bearing_outliers)

        
            .pipe(select_columns, 
                  'passenger_count',
                  'pickup_longitude',  
                  'pickup_latitude',   
                  'dropoff_longitude', 
                  'dropoff_latitude',
                  'month',
                  'day_of_week',
                  'hour',
                  'manhattan',
                  'haversine',
                  'bearing'
                 )
        )      
        
        y = remove_manhattan_hav_and_bearing_outliers(data)['duration']
        
    return X, y

In [20]:
# Train
X_train2, y_train2 = process_data_gm3(all_m_train_df)
X_val2, y_val2 = process_data_gm3(all_m_val_df)
guided_model_3 = lm.Ridge(fit_intercept=True)
guided_model_3.fit(X_train2, y_train2)

# Predict
y_train_pred2 = guided_model_3.predict(X_train2)
y_val_pred2 = guided_model_3.predict(X_val2)

/srv/conda/envs/data100/lib/python3.6/site-packages/pandas/core/generic.py:4388: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/srv/conda/envs/data100/lib/python3.6/site-packages/pandas/core/generic.py:4389: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


Let's find MAE

In [21]:
assert 200 <= mae(y_val_pred2, y_val2) <= 300
print("Validation Error: ", mae(y_val_pred2, y_val2))

Validation Error:  250.593999008


## Submission

You're almost done!

Before submitting this assignment, ensure that you have:

1. Restarted the Kernel (in the menubar, select Kernel$\rightarrow$Restart & Run All)
2. Validated the notebook by clicking the "Validate" button.

Then,

1. **Submit** the assignment via the Assignments tab in **Datahub** 
1. **Upload and tag** the manually reviewed portions of the assignment on **Gradescope**